In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup headless Chrome
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ganti path dengan lokasi chromedriver kamu
driver = webdriver.Chrome(options=options)

In [5]:
# Ganti URL di bawah dengan URL web target sebenarnya
# url = "https://mlbb.gg/statistics"
url = "https://mobile-legends.fandom.com/wiki/List_of_heroes"

driver.get(url)

# Fungsi untuk scroll ke bawah sampai mentok
def scroll_to_bottom(driver, pause_time=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll ke bawah
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Tunggu loading selesai
        time.sleep(pause_time)
        
        # Cek tinggi halaman setelah scroll
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Sudah sampai bawah
        last_height = new_height

# Scroll hingga semua data termuat
scroll_to_bottom(driver)

# Tunggu tambahan jika perlu
time.sleep(3)

# Ambil halaman setelah scroll selesai
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
# Cari elemen <table>
table = soup.find('table')

# Ambil header (thead)
thead = table.find('thead')
header = []
if thead:
    ths = thead.find_all('th')
    for th in ths:
        # Ambil teks dalam <th> (bisa dari dalam div juga)
        text = th.get_text(strip=True)
        header.append(text)
else:
    print("Tidak ditemukan elemen <thead>")

# Ambil isi baris (tbody)
tbody = table.find('tbody')
rows_data = []
if tbody:
    rows = tbody.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        row_text = [col.get_text(strip=True) for col in cols]
        rows_data.append(row_text)
else:
    print("Tidak ditemukan elemen <tbody>")

# Tampilkan hasil (opsional: simpan ke DataFrame atau CSV)
df = pd.DataFrame(rows_data, columns=header)
print(df)

# Simpan ke CSV (opsional)
# df.to_csv("data/data_heroes.csv", index=False)
# df.to_csv("data/data_heroes_2.csv", index=False)

## Integrasi Data Lama

In [ ]:
# data_fix = pd.read_csv('../data/fix.csv', encoding='windows-1252')
# data_hero_1 = pd.read_csv('../data/data_heroes.csv', encoding='windows-1252')

In [ ]:
# # Lakukan merge dengan outer agar semua data masuk (baik dari kiri atau kanan)
# data_fix_cp = data_fix.merge(
#     data_hero_1[['Hero', 'Win rate', 'Pick rate', 'Ban rate']],
#     how='outer',
#     left_on='Hero Name',
#     right_on='Hero'
# )

# # Jika kolom 'Hero Name' kosong, gunakan nilai dari kolom 'Hero'
# data_fix_cp['Hero Name'] = data_fix_cp['Hero Name'].combine_first(data_fix_cp['Hero'])

# data_fix_cp.drop(columns=['Hero','Win Rate (%)', 'Popularity (%)', 'Ban Rate (%)'], inplace=True, axis=1)

# # Ganti nama kolom hasil merge
# data_fix_cp.rename(columns={
#     'Win rate': 'Win Rate (%)',
#     'Pick rate': 'Popularity (%)',
#     'Ban rate': 'Ban Rate (%)',
# }, inplace=True)

# # # Hapus kolom 'Hero' jika sudah tidak perlu
# # data_fix_cp.drop(columns='Hero', inplace=True)

# # # Optional: Reset index jika perlu
# data_fix_cp.reset_index(drop=True, inplace=True)

,Hero Name,Tier
113,Uranus,D
2,Aldous,D
89,Minsitthar,C
77,Lesley,D
57,Hylos,C
62,Joy,S
119,X.Borg,D
114,Vale,C
10,Aulus,C
107,Silvanna,D


## Pengkalian Kolom Baru

In [ ]:
# data_fix_cp['Ban Rate (%)'] = (data_fix_cp['Ban Rate (%)'] * 100).astype(int)
# data_fix_cp['Win Rate (%)'] = (data_fix_cp['Win Rate (%)'] * 100).astype(int)
# data_fix_cp['Popularity (%)'] = (data_fix_cp['Popularity (%)'] * 100).astype(int)

In [ ]:
# data_fix_cp.to_csv('../data/new_fix.csv', index=False)

## Update Tier Data

In [2]:
fix_updated = pd.read_excel('../data/new_fix.xlsx')
data_hero_1 = pd.read_csv('../data/data_heroes.csv', encoding='windows-1252')

In [3]:
data_fix_cp = fix_updated.merge(
    data_hero_1[['Hero','Tier']],
    how='outer',
    left_on='Hero Name',
    right_on='Hero'
)

data_fix_cp['Hero Name'] = data_fix_cp['Hero Name'].combine_first(data_fix_cp['Hero'])

data_fix_cp.drop(columns='Tier Old', inplace=True)

data_fix_cp.reset_index(drop=True, inplace=True)

In [4]:
data_fix_cp.to_csv('../data/fix.csv', index=False)

In [5]:
data_fix_cp.isnull().sum()

Hero Name                     0
Hero ID                       0
Roles                         0
Specialties                   0
Recommended Lane              0
Second Lane                 108
Release year                  0
Meta                          0
Difficulty                    0
Durability Rating             0
Ability Effects Rating        0
Offense Rating                0
Scaling Rating                0
Cooldown Rating               0
Item Dependency Rating        0
Mobility Rating               0
Crowd Control Rating          0
Base Stats Growth Rating      0
Ultimate Impact Rating        0
Column1                       0
Gameplay                      0
Sinergi                       0
Win Rate (%)                  0
Popularity (%)                0
Ban Rate (%)                  0
Hero                          0
Tier                          0
dtype: int64